# Активность пользователей в поиске VK Видео 
- Автор: Пулькина Анастасия
- Дата: 02.04.2025

<div class="alert alert-info">

### Цели и задачи проекта
    
    
</div>

**Цель проекта** - анализ данных об активности пользователей в поиске VK Видео. 

**Задачи**:
 1. Знакомство с данными: загрузка, предобработка (обработка типа данных и пропусков)  
 2. Поиск точек роста продукта: в каких случаях поиск показывает плохое качество.
 3. Используя визуализацию показать проблемные случаи  
 4. Краткие выводы 

Датасет `analytics_test.csv` содержит информаию о пользователях и их запросах. В датасете представленны следующие данные: 
- `user_id`	- идентификатор пользователя
- `sex`	- пол пользователя
- `age`	- возраст
- `platform` - платформа	
- `query_id` - идентификатор поиска	
- `query_text` - текст введенного запроса	
- `query_type` - тема запроса  	
- `event` - действие, которое совершает пользователь (`tap`, `like`, `subscribe`, `view`, `share`, `search`)	
- `click_position` - номер позиции, по которой кликает пользователь	
- `doc_type`	- тип документа по которому совершется событие `event`
- `view_time` - время просмотра

Для начала необходимо определить, что будем считать плохим качесвтом поиска. Критерии могут быть следующими: 
- Низкая релевантность выдачи (приходится кликать на более далекие позиции `click_position`, чтобы найти ответ)
- Короткое время просмотра `view_time`, что может говорить о том, что пользователь не вовлекся и быстро ушел со страницы 
- Мало действий `event`, что готовит о низкой вовлеченности пользователя
- Повторные поиски `search` в `event` по похожему запросу


<div class="alert alert-info">
    
### Содержимое проекта
</div>

1. [Загрузка и предобработка данных](#download_data)
2. [Исследовательский анализ данных](#eda)
3. [Итоговый вывод](#summary)
---

<div class="alert alert-info">
    
## Загрузка, знакомство с данных и их предобработка
<a id='download_data'></a>
</div>

In [1]:
# Импортирую необходимые библиотеки 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
!pip install phik
from phik import phik_matrix

In [3]:
!pip install matplotlib-venn
from matplotlib_venn import venn3

In [13]:
import requests

In [14]:
# Загружаю датасет

public_key = "https://disk.yandex.ru/d/GruOijWHnJE1og"  

# 1. Запрашиваем у API Яндекс.Диска прямой (download) URL для чтения файла
api_endpoint = "https://cloud-api.yandex.net/v1/disk/public/resources/download"
params = {"public_key": public_key}
response = requests.get(api_endpoint, params=params)
response.raise_for_status()  

download_url = response.json()["href"]
# Теперь download_url — это прямой адрес к самому файлу без CAPTCHA

# 2. Считываем CSV напрямую из полученного download_url
df = pd.read_csv(download_url)

df.head()

,Unnamed: 0,user_id,sex,age,platform,query_id,query_text,query_type,event,click_position,doc_type,view_time
0,0,96422511,female,68,android,89656828,передачи о новых технологиях,show,share,6,playlist,0
1,1,45247030,male,62,android_tv,89784877,автопрому новости,news,search,0,NaN,0
2,2,79572413,male,12,web,10555855,логистическое законодательство,news,search,0,NaN,0
3,3,821317,female,58,android_tv,72054848,банковские премии новости,news,search,0,NaN,0
4,4,6206023,female,24,android,20838049,вики шоу аманда,show,subscribe,7,author,0


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      20000 non-null  int64 
 1   user_id         20000 non-null  int64 
 2   sex             20000 non-null  object
 3   age             20000 non-null  int64 
 4   platform        20000 non-null  object
 5   query_id        20000 non-null  int64 
 6   query_text      20000 non-null  object
 7   query_type      20000 non-null  object
 8   event           20000 non-null  object
 9   click_position  20000 non-null  int64 
 10  doc_type        15042 non-null  object
 11  view_time       20000 non-null  int64 
dtypes: int64(6), object(6)
memory usage: 1.8+ MB


In [6]:
# ищу пропуски 
df_missing = df.copy() # Буду работать с копием        
missing = pd.DataFrame({
    'Кол-во пропусков': df_missing.isnull().sum(), 
    'Доля пропусков': df_missing.isnull().mean().round(4)})
           
missing.sort_values(by='Кол-во пропусков', ascending=False).style.background_gradient(cmap='coolwarm')

/Users/a1234/anaconda3/lib/python3.11/site-packages/pandas/io/formats/style.py:3618: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(gmap) if vmin is None else vmin
/Users/a1234/anaconda3/lib/python3.11/site-packages/pandas/io/formats/style.py:3619: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(gmap) if vmax is None else vmax


,Кол-во пропусков,Доля пропусков
"<!doctype html><meta name=""viewport"" content=""width=device-width",0.000000,nan
"sitekey:""""",0.000000,nan
"linksVisible:""yes""}",0.000000,nan


In [7]:
# Типы данных соответствуют, проверим, можно ли снизить их разрядность
for column in ['user_id', 'age', 'query_id', 'click_position', 'view_time']:
    df[column] = pd.to_numeric(df[column], downcast='integer')    
    print(f'Тип данных столбца {column} после преобразования:', df[column].dtype)

KeyError: 'user_id'

In [ ]:
# Посмотрим на уникальные значения и их число 
for column in df.columns:
    print(f'Число уникальных значений в столбце {column}:')
    print(df[column].sort_values().nunique())
    print(f'Уникальные значения в столбце {column}:')
    print(df[column].sort_values().unique())
    print()

In [ ]:
# посмотрим на пользователей, у которых возраст стоит 0. 
df[df['age']==0]

In [ ]:
# Посмотрим на распределение значений с возрастом 0 по признаку пола
df[df['age']==0]['sex'].value_counts()

In [ ]:
# Посмотрим на пользователей, у которых в столбце `doc_type` нет данных
df[df['doc_type'].isna()]

In [ ]:
# Посмотрим на распределение значений столбца 'click_position' в запросах, где нет данных в столбце 'doc_type'
df[df['doc_type'].isna()]['click_position'].value_counts()

In [ ]:
# Посмотрим, какие действия производят пользователи6 если в 'doc_type' нет данных
df[df['doc_type'].isna()]['event'].value_counts()

In [ ]:
# Посмотрим на распределение значений столбц view_time' в запросах, где нет данных в столбце 'doc_type'
df[df['doc_type'].isna()]['view_time'].value_counts()

In [ ]:
# и тоже самое сделаем для типа запросов
df[df['doc_type'].isna()]['query_type'].value_counts()

In [ ]:
# и платформы
df[df['doc_type'].isna()]['platform'].value_counts()

In [ ]:
df[df['view_time']==0]['event'].value_counts()

In [ ]:
df[df['click_position']==0]['event'].value_counts()

Если в столбце `doc_type` нет данных, то пользователи осуществляют только поиск. И в столбцах `click_position` и `view_time` данные равны нулю. Значение 0 в столбце `click_position` видимо является заглушкой, согнализирующей об отсутствии последующего действия, так как в описании задания сказано, что первый элемент выдачи - 1. Однако значение 0 в столбцах `click_position` и `view_time` встречается не только для поиска но и для других действий. 

In [ ]:
df.head()

In [ ]:
# Проверяю наличие полных явных дубликатов в датафрейме 
print('Количество дубликатов:', df.duplicated().sum())

### Промежуточный вывод 

Предварительное знакомство с датасетом `analytics_test.csv` показало, что датасет содержит 20000 строк и 12 столбцев. Названия столбцев корректные, соответствуют информации, содержащейся в ней. В столбце `doc_type` содержится 4958 пропусков, что составляет 25% от общего числа сторок. Это пользователи, которые осуществляли только поиск. Для них значени в столбцах `click_position` и `view_time` равны 0. Распределение по теме запросов и по используемой платформе примерно равномерное.   
В столбце `age`, возраст встречается 0, что может быть заглушкой, если пользователь, например, не указал свой возраст (но мне кажется, что при регистрации в ВК, указание возраста обязательно) Следующая цифра по возрастанию в столбце возраст - 10. Пол у пользователей с возрастом 0 по большей части оставлен анонимным, но встречаются как мужчины, так и женщины. Возможно, есть какая-то опция о приватности личных данных. Эти данные требуют уточнения.  
При предварительном просмотре также понизила размерность целочисленных данных, явных полных дубликатов не обнаружила. 


<div class="alert alert-info">
    
## Исследовательский анализ данных
<a id='eda'></a>
</div>

In [ ]:
df.head()

<div style="background-color:lightblue;">
    
### Распределение событий 

</div>

In [ ]:
# Посмотрим на распределение событий 'event' в нашем датасете 
pd.DataFrame({
     'Кол-во событий': df['event'].value_counts(), 
    'Доля событий': df['event'].value_counts(normalize=True).round(2)})


In [ ]:
# визуализируем его в долях
grouped_event = df['event'].value_counts(normalize=True).sort_values(ascending = True)

grouped_event.plot(kind='barh',
                   title='Популярность действий пользователей',
                   legend=False,
                   ylabel='Название действия',
                   xlabel='Число действий',
                   edgecolor='grey',
                   rot=0,
                   figsize=(8, 5),
                   color='lightblue')

plt.grid()
plt.show()

Больше всего пользователи кликают (33%) и что-то ищут (25%). 

In [ ]:
# Посмотрим на разброс значений в столбце 'click_position' в зависимости от происходящего события
plt.figure(figsize=(7, 4))
sns.boxplot(x= 'event', 
            y='click_position', 
            data=df,
            palette='Blues', 
            showmeans=True  
            ).set(title = 'Разброс значений кликабельных позиций \n в зависимости от действия', 
                             xlabel = 'Действие', 
                             ylabel = 'Номер позиции'
                             )   

plt.xticks(rotation=45)
plt.grid()
plt.show()

Наименьшая позиция в выдаче поиска для поиска. Это мы уже знает, что оно равно 0. Остальные распределены относительно равномерно.

<div style="background-color:lightblue;">
    
### Распределение позиции документа в выдаче поиска по которму совершено событие и времени просмотра

</div>

In [ ]:
# Построим диаграмму размаха для столбца 'click_position'
plt.figure(figsize = (10, 3))
df.boxplot(column = 'click_position', vert = False, showfliers=True)
plt.title('Распределение значений номера позиции документа \n в выдаче поиска')
plt.xlabel('Позиция документа, по которой совершается событие')


plt.show()

Есть выбросы, но я бы не стала их исключать из анализа, они не выглядят на мой взгляд подозрительно.

In [ ]:
plt.figure(figsize=(7, 4))
sns.boxplot(x='query_type', 
            y='click_position', 
            data=df,
            palette='Blues',
            showmeans=True  
            ).set(title = 'Разброс значений кликабельных позиций \n в зависимости от темы запроса', 
                             xlabel = 'Тема запроса', 
                             ylabel = 'Номер позиции'
                             )   

plt.xticks(rotation=45)
plt.grid()
plt.show()

Самый большой разброс позиции документа в выдаче поиска для новостных запросов. Для них же отмечено самое больше значение и самые высокие среднее и медиана. То есть при чтении новостей люди часто открывают не первую позицию в выдаче. Это может говорить о том, что им не нравится то, что предлатается в начале.

In [ ]:
# Посмотрим на средние значения 'click_position' и 'view_time' при разбивке по темам запроса 'query_type'

grouped_type = df.groupby('query_type').agg({
    'click_position': 'mean', 
    'view_time': 'mean'
}).sort_values('click_position', ascending=True)
# Построим столбчатые диаграммы на двух осях
fig, ax1 = plt.subplots(figsize=(8, 5))
ax2 = ax1.twiny()  

bar_width = 0.35
index = np.arange(len(grouped_type.index))

# Номер позиции по которой кликает пользователь
ax1.barh(index - bar_width/2, grouped_type['click_position'],
         height=bar_width, color='blue', alpha=0.7, label='Позиция документа в выдаче поиска', edgecolor='grey')
ax1.set_xlabel('Средний номер позиции', color='blue')

# Время просмотра
ax2.barh(index + bar_width/2, grouped_type['view_time'],
         height=bar_width, color='lightblue', alpha=0.7, label='Время просмотра (с)', edgecolor='grey')
ax2.set_xlabel('Среднее время просмотра', color='lightblue')

# Общие настройки
ax1.set_yticks(index)
ax1.set_yticklabels(grouped_type.index)
ax1.legend(loc='lower right')
ax2.legend(loc='upper right')
plt.grid()

plt.tight_layout()

In [ ]:
pd.pivot_table(df,
                index='query_type',
                values=['view_time', 'click_position'],
                aggfunc=['mean', 'max', 'count'])

Ожидаемо, самое большое среднее время просмотра  и максимальное значение у фильмов. Средняя и максимальная наибольшая позиция документа, по которой кликают, - у новостей. При этом и время просмотра для новостной тематики не высокое.  Здесь нужно отметить, что число наблюдений по группам query_type одинаковое. 

Посмотрим, есть ли какая-то зависимость типа документа, кликабельной позиции и временем просмотра с другими данными.

In [ ]:
# Построим корреляционную матрицу
cor_matrix = df.phik_matrix()
plt.figure(figsize=(12,6))
data_heatmap = cor_matrix[['click_position', 'doc_type', 'view_time']].sort_values(by='click_position', ascending=False)
sns.heatmap(data_heatmap,
           annot = True, 
           fmt='.2f', 
           cmap = 'coolwarm',
           cbar = False
           )

plt.title('Тепловая карта коэффициента phi_k для типа документа')
plt.xlabel('Тип документа')

plt.show()

Есть умеренная положительная корреляция позиции документа в поиске с темой, текстом запроса и действием; типа документа с темой запроса и текстом запроса; и времени просмотра с событием.  

Тему с наибольшим средним числом позиции документа в поиске мы уже определели (новости). Теперь посмотрим на текст запросов и действия в запросах с далекой кликабельной позицией. Также посмотрим на другие выявленные корреляции. 

<div style="background-color:lightblue;">
    
### Выявление проблемных запросов. Гипотезы.

</div>



**1. Если пользователь совершает действие по позиции документа в выдаче поиска больше среднего значения, то выдача нерелевантна.**  

In [ ]:
# Найдем среднее значение позиции документа в выдаче поиска, по которому совершается действие
df['click_position'].mean().round(2)

Среднее значение для числа кликов - 6. Будем считать это пороговым значением. Отфильтруем наш датасет по этой границе и выведем только топ-10. Я хочу сразу посмотреть на время посмотра. Если оно маленькое - значит человеку было не интересно и он не перешел ни на одну страницу поиска, поиск был нерелевантен. 

In [ ]:
bad_clicks = df[df['click_position'] > 6].groupby('query_text').agg({
    'click_position': 'mean', 
    'view_time': 'mean'
}).sort_values('click_position', ascending=False).head(10)
bad_clicks

Все запросы связаны с новостной тематикой.  
Дополнительно отфильтрую датасет, оставив в нем те запросы, в которых время просмотра равно 0. Большое число кликов и нулевое время просмотра могут значить нерелевантную выдачу. 

**2. Если время просмотра равно нулю и кликабельная позиция выше средней - поиск по запросу не релевантен**

In [ ]:
bad_clicks_views = df[
    (df['click_position'] > 6)&
    (df['view_time'] ==0)
].groupby('query_text')['click_position'].mean().sort_values(ascending=False).head(10)

bad_clicks_views.plot(kind='barh', 
                        title='Топ-10 запросов с высокой позицией документа в выдаче\n и нулевым временем просмотров', 
                        color='lightblue', 
                        edgecolor='grey')

plt.xlabel('Средняя позиция документы в выдаче')
plt.ylabel('Запрос')
plt.grid()
plt.show()

Также все запросы остались с новостной тематикой.  Продолжим изучение коротких просмотров. 

**3. Если время просмотра короткое (равно 0), то результаты поиска скорее всего не заинтересовали**

In [ ]:
bad_views_text = df[df['view_time'] ==0].groupby('query_text')['view_time'].count().sort_values(ascending=False).head(10)

bad_views_text.plot(kind='barh', 
                        title='Топ-10 запросов с нулевым временем просмотров', 
                        color='lightblue', 
                        edgecolor='grey')
plt.xlabel('Число нулевых просмотров')
plt.ylabel('Запрос')
plt.grid()
plt.show()

Наибольшее число запросов, в которых время просмотра равно 0, связано с фильмами. 

In [ ]:
# посмотрим, какие темы для поиска приводят к нулевым просмотрам
bad_views_type = df[df['view_time'] ==0]['query_type'].value_counts().sort_values(ascending = True)

bad_views_type.plot(kind='barh',
                   title='Количество поисков по тематике при нулевом времени просмотра',
                   legend=False,
                   ylabel='Тема поиска',
                   xlabel='Число поисков',
                   edgecolor='grey',
                   color='lightblue')

plt.grid()
plt.show()

Но если смотреть темы отдельно, кажется, что нет лидеров и аутсайдеров.

In [ ]:
# посмотрим, какие события сопровождаются нулевыми просмотрами
bad_views_event = df[df['view_time'] ==0]['event'].value_counts().sort_values(ascending = True)

bad_views_event.plot(kind='barh',
                   title='Действия пользователей при нулевом просмотре',
                   legend=False,
                   ylabel='Название действия',
                   xlabel='Число действий',
                   edgecolor='grey',
                   color='lightblue')

plt.grid()
plt.show()

Нулевые просмотры даже приводят к тому, что пользователи лайкают, подписываются и делятся найденным контентом (это было замечено и выше). Однако наиболее популярное действие, не сопровождающееся просмотром, - просто кликанье и поиск. 

In [ ]:
# Посмотрим, как действия с нулевым времением распределены по темам
pd.pivot_table(df[df['view_time'] ==0],
                        values='query_id', 
                        index='query_type', 
                        columns='event', 
                        aggfunc='nunique')

Чаще всего днействия с нулевым временем встречаются при кликании по теме новости. Дальше шшоу, фильмы и музыка. Меньше всего нулевых просмотров при шеринге музыкой и подпиской на новости.

In [ ]:
# тогда еще посмотрим, в каком типе контента больше всего встречаются нулевые просмотры
bad_views_doc = df[df['view_time'] ==0]['doc_type'].value_counts().sort_values(ascending = True)

bad_views_doc.plot(kind='barh',
                   title='Популярность типа контента среди нулевых просмотров',
                   legend=False,
                   ylabel='Тип контента',
                   xlabel='Число запросов',
                   edgecolor='grey',
                   color='lightblue')

plt.grid()
plt.show()

Наиболее характерно нулевое время просмотра для клипов. Наименее характерно - для плейлистов.

**4. Если мало лайков и шерингов, то результаты поиска не вызывают интерес и вовлеченность**

In [ ]:
# Посмотрим на топ-10 запросов с низкой вовлеченностью
low_intrest = df[df['event'].isin(['like', 'share'])]['query_text'].value_counts()
print('\nТоп-10 запросов с низкой вовлечённостью:\n', low_intrest.tail(10))

In [ ]:
# Посмотрим, какая тема вывывает наименьшую вовлеченность
low_intrest_type = df[~df['event'].isin(['like', 'share'])]['query_type'].value_counts().sort_values(ascending = True)

low_intrest_type.plot(kind='barh',
                   title='Темы с низкой вовлечённостью',
                   legend=False,
                   ylabel='Тема запроса',
                   xlabel='Число запросов',
                   edgecolor='grey',
                   color='lightblue')

plt.grid()
plt.show()

Темы с низкой вовлеченностью - новости и фильмы. Им меньше всего ставят лайков и реже делятся. 

In [ ]:
# И какой тип контента менее интересен
low_intrest_doc = df[~df['event'].isin(['like', 'share'])]['doc_type'].value_counts().sort_values(ascending = True)

low_intrest_doc.plot(kind='barh',
                   title='Типы контента с низкой вовлечённостью',
                   legend=False,
                   ylabel='Тип контента',
                   xlabel='Число запросов',
                   edgecolor='grey',
                   color='lightblue')

plt.grid()
plt.show()

Клипы и видео вызывают меньше всего интереса, им реже ставят лайки и ими реже деляться с друзьями.

**5. Пересечение поисковых текстов**

In [ ]:
# Построим диаграмму Венна
set_intrest = set(df[df['event'].isin(['like', 'share'])]['query_text'])
set_view = set(df[df['view_time'] ==0]['query_text'])
set_click = set(df[df['click_position'] > 6]['query_text'])

venn3([set_intrest, set_view, set_click], set_labels=('Не интересно', 'Нет просмотров', 'Высокая позиция клика в поиске'))


plt.title('Пересечение текстовых запросов')
plt.show()

1125 запросов попадают во все использованные фильтрации. То есть у 1125 запросов из 20000 нулевое время просмотра, номер позиции клика больше 6 и их не лайкают и ими не делятся пользователи. Это что-то около 5%. Не уверена, что это критично. 


### Промежуточный вывод

В результате исследовательского анализа посмотрела на распределение числовых данных, построила их боксплоты. Посмотрела на корреляцию некоторых признаков, которые на мой взгляд могли бы говорить о релевантности контента в поиске. Были выдвинуты следующие гипотезы: 
1. Если пользователь совершает действие по позиции документа в выдаче поиска больше среднего значения, то выдача не релевантна  
2. Если время просмотра равно нулю и кликабельная позиция выше средней - поиск по запросу не релевантен  
3. Если время просмотра короткое (равно 0), то результаты поиска скорее всего не заинтересовали  
4. Если мало лайков и шерингов, то результаты поиска не вызывают интерес и вовлеченность  

Для проверки гипотез были посчитаны и визуализированы число значений, среднее при разбивке на различные группы. 

Наибольшая позиция документа в выдаче поиска оказалась характерна для новостных поисков. Наибольшее число запросов, в которых время просмотра равно 0, было связано с фильмами. Но если смотреть наибольшую позицию документа в выдаче поиска и нулевое время просмотра, то нерелевантными остаются новостные запросы. Ими меньше делятся, им меньше ставят лайков (отсутствие лайков конечно может быть связано с тем, что новость в негативном ключе, а не потому что она неинтересная). 

<div class="alert alert-info">
    
## Итоговый вывод
<a id='summary'></a>
</div>

- Поиск показывает плохое качесвто в случае новостных запросов. 
- Пользователи открывают в среднем 11 позицию документа в выдаче поиска. Возможно, проблема в алгоритме ранжирования например.
- Пользователи быстро покидают платформу, может быть низкое качество контента по темам шоу и новостей (по этим темам наиболее часто встречается время просмотра равное 0). 
- Низкая вовлеченность пользователей при поиске новостей и фильмов. Им редко ставят лайки и редко делятся с друзбями. Тоже касается клипов и видео. 


Что делать дальше?
- A/B-тестирование новых алгоритмов ранжирования для новостных запросов.
- Опрос пользователей, чтобы понять, почему они не вовлекаются.


PS буду благодарна за обратную связь